In [1]:
# Standard libraries
import os

# Third-party libraries
import geopandas as gpd
import pandas as pd


In [2]:
# Paths
DATA_DIR = os.path.join(os.getcwd(), "data")
GEO_DIR = os.path.join(DATA_DIR, "intermediate/spatial")
OUT_DIR = os.path.join(DATA_DIR, "processed")

# Constants
OSGB36 = 27700


In [3]:
def format(df):
    """Rename some columns and drops some others"""
    return (
        df.rename(columns={"geo_region": "name", "region": "id"})
        .sort_values(by="id", axis=0)
        .reset_index(drop=True)
    )


def read(fp):
    """Reads and prepares the geoJson"""
    df = gpd.read_file(fp).pipe(format)
    return df


def save(df, file_name):
    df.to_file(os.path.join(OUT_DIR, file_name + "-geom.geojson"), driver="GeoJSON")
    df.drop(columns=["x_coord", "y_coord", "geometry"]).to_csv(os.path.join(OUT_DIR, file_name + ".csv"), index=False)


In [5]:
# Regions
region_fp = os.path.join(GEO_DIR, "ukcp18-uk-land-region-hires-osgb36.geojson")
region = read(region_fp)
save(region, "region")


In [6]:
# Countries
country_fp = os.path.join(GEO_DIR, "ukcp18-uk-land-country-hires-osgb36.geojson")
iso = pd.read_csv(os.path.join(GEO_DIR, "iso-3166-2-gb.csv"))
country = read(country_fp).drop(columns="id")
country = pd.merge(country, iso, on="name")
save(country, "country")
